# ベルマン方程式

本章，価値関数/行動価値関数が満たす再帰的な性質を基にした **ベルマン方程式** を紹介します．
この方程式は，現在の状態の価値$V(S_t)$と次の状態の価値$V(S_{t+1})$の関係に着目することにより得られる方程式で，
様々な強化学習アルゴリズムで利用されています．


## ベルマン方程式

価値関数/行動価値関数に関するベルマン方程式は以下のようになります．

$$V$$
